In [1]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
import geopy.distance
from icecream import ic
import functools
from dataclasses import dataclass
logging.basicConfig(level=logging.DEBUG)

In [2]:
CITIES_ROOTPATH = "cities/"
CITIES=["vanuatu.csv", "italy.csv", "russia.csv", "us.csv", "china.csv"]

In [3]:
cities = pd.read_csv(CITIES_ROOTPATH+CITIES[0],header=None, names=["name","lat","lon"])

In [4]:
dist_matrix = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

In [5]:
# I initialize the rng with repeatable values provided by the instance:
# first 10 cities in list format, rounded to int and in absolute value
rng = np.random.Generator(np.random.PCG64(cities.lat.head(10).astype(int).abs().to_list()))

## GA implementation
I my implementation i only tweak on permutation, as:
- the problem is meaningless if not all item are taken
- there is no meaning in repeated visit of nodes, as the direct link should be the fastest being the graph *fully conected*

In [6]:
def randomGenome():
    genome = np.arange(len(cities))
    rng.shuffle(genome)
    return genome
"""
Random initialization of the genome
"""

'\nRandom initialization of the genome\n'

In [7]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

@counter
def genome_cost(genome:np.ndarray):
    """Cost function for the given solution, including the call counter decorator"""
    total_len=0
    for cpair in zip(genome,np.roll(genome,-1)):
        total_len += dist_matrix[cpair[0], cpair[1]]
    return total_len

def genome_fitness(genome:np.ndarray):
    return genome_cost(genome)



class Individual:
    __genome: np.ndarray=None
    __fitness:float=None

    def __init__(self,genome:np.ndarray=None,fitness=None):
        if genome is not None:
            if fitness is not None:
                self.__genome = genome.copy()
                self.__fitness = fitness
            else:
                self.setGenome(genome)

    @staticmethod
    def getRandom():
        return Individual(randomGenome())
    
    
    def setGenome(self,genome:np.ndarray):
        self.__genome = genome.copy()
        self.__fitness = genome_fitness(self.__genome)

    def genome(self)->np.ndarray:
        return self.__genome.copy()

    def fitness(self):
        return self.__fitness
    
    def copy(self):
        return Individual(self.__genome,self.__fitness)

Now i use the `Individual` class to define on it my mutation operators; i defined 3 different mutation operators:
- `swap_mutation(parent,step=1)`: choose a locus and swap its allele to the one at distance provided by the parameter `step` (considering circularity); `abs(step)` must be l.e. the size of the parent genome;
- `inverse_mutation(parent,size=3)`: randomly choose 2 loci where the region defined by them (them included, also considering circularity) has size=`size` and invert the alleles contained by them; require a `size` parameter, representing the size of the inverted region, that must be between `2` and the size of the parent genome;
- `insertion_mutation(parent,num_skips=2)`: choose a random locus and move its alleles near another, shifting the others to provide insertion; require a `num_skips` parameter, representing the number of loci skipped (*going left*) by the moving allele;

In [8]:
def swap_mutation(parent:Individual,*,step=1):
    parent_genome = parent.genome()
    if step==0:
        return
    elif abs(step)>parent_genome.size:
        raise(f"Cannot use a step of {step} in a {parent_genome.size}-long array")
    loc1 = rng.integers(parent_genome.size)
    loc2 = loc1+step
    if loc2>parent_genome.size-1:
        loc2 = loc2 % parent_genome.size # circular index
    parent_genome[loc1],parent_genome[loc2] = parent_genome[loc2],parent_genome[loc1]
    return Individual(parent_genome)

def inverse_mutation(parent:Individual,size=3):
    parent_genome = parent.genome()
    if size<2:
        return
    elif size > parent_genome.size:
        raise(f"Inverting size {size} cannot be applied to a {parent_genome.size}-long array")
    
    """
    Instead of randomly choosing the first locus, i provide circularity by providing a random roll to the array and then operating on it:
    in this case, choosing as 0 the first index is equivalent; finally the original order is restored by reversing the roll before returning;
    """
    rollnum = rng.integers(parent_genome.size-1) 
    circ_genome = np.roll(parent_genome,-rollnum)

    # select the subset of the given size, invert it and substitute it to the genome in the curent rotation
    last_ind = size-1 # last index is simply size-1 (sure not to go out of borders)
    inv_subset = circ_genome[:last_ind+1][::-1] 
    circ_genome[:last_ind+1] = inv_subset

    # restore original order and return
    final_genome = np.roll(circ_genome,rollnum)
    return Individual(final_genome)

def insertion_mutation(parent:Individual,num_skips=2):
    parent_genome = parent.genome()

    if num_skips<1:
        return
    elif num_skips>parent_genome.size-2:
        raise(f"It is impossible to skip more than {parent_genome.size-2} items for insertion in a {parent_genome.size}-long array; you tried skipping {num_skips}")
    
    # perform removal
    rem_ind = rng.integers(parent_genome.size-1) # random index to remove from
    tmp = parent_genome[rem_ind] # save value
    parent_genome = np.delete(parent_genome,rem_ind)

    # perform insertion and return
    ins_ind = rem_ind-num_skips
    parent_genome = np.insert(parent_genome,ins_ind,tmp)
    return Individual(parent_genome)

Now i do the same, using the `Individual`, but to define on it my **xover** operators; i defined 3 different mutation operators:
- `PMX(parent1,parent2,slice_size=2)`: Partially mapped crossover, with given size=`slice_size` (that must be between l.e. than the size of the genome of the parents) representing the size of the contiguous slice fully copied from the first parent;
- `OX(parent1,parent2,slice_size=2)`: Order Crossover, with given size=`slice_size` (that must be between l.e. than the size of the genome of the parents) representing the size of the contiguous slice fully copied from the first parent;
- `CX(parent1,parent2)`: Cicle Crossover;

In [61]:
def PMX(parent1:Individual,parent2:Individual,*,slice_size=2):
    sz = slice_size
    pg1 = parent1.genome()
    pg2 = parent2.genome()
    if sz<2:
        return
    elif pg1.size != pg2.size:
        raise(f"Length of the genomes of the 2 parents must match, instead you provided parents with g.l. {pg1.size} and {pg2.size}")
    elif sz > pg1.size:
        raise(f"Slice size {sz} cannot be applied to {pg1.size}-long arrays")
    
    psz = pg1.size
    
    """
    Instead of randomly choosing the first locus, i provide circularity by providing a random roll to the array and then operating on it:
    in this case, choosing as 0 the first index is equivalent; finally the original order is restored by reversing the roll before returning;
    """
    rollnum = rng.integers(psz-1) 
    rot_pg1 = np.roll(pg1,-rollnum)
    rot_pg2 = np.roll(pg2,-rollnum)

    rot_sg = rot_pg1.copy() # create son genome (rotated)

    # select the subset of the given size from the first parent and create the mapping
    mapping = dict()
    last_ind = sz-1 # last index is simply sz-1 (sure not to go out of borders)
    for i in range(last_ind+1):
        mapping[rot_pg1[i]]=rot_pg2[i]
    for i in range(last_ind+1,psz):
        if rot_pg2[i] in mapping:
            rot_sg[i] = mapping[rot_pg2[i]]
        else:
            rot_sg[i]=rot_pg2[i]
    
    son_genome = np.roll(rot_sg,rollnum)
    return Individual(son_genome)

def OX(parent1:Individual,parent2:Individual,slice_size=2):
    sz = slice_size
    pg1 = parent1.genome()
    pg2 = parent2.genome()
    if sz<2:
        return
    elif pg1.size != pg2.size:
        raise(f"Length of the genomes of the 2 parents must match, instead you provided parents with g.l. {pg1.size} and {pg2.size}")
    elif sz > pg1.size:
        raise(f"Slice size {sz} cannot be applied to {pg1.size}-long arrays")
    
    psz = pg1.size
    
    """
    Instead of randomly choosing the first locus, i provide circularity by providing a random roll to the array and then operating on it:
    in this case, choosing as 0 the first index is equivalent; finally the original order is restored by reversing the roll before returning;
    """
    rollnum = rng.integers(psz-1) 
    rot_pg1 = np.roll(pg1,-rollnum)
    rot_pg2 = np.roll(pg2,-rollnum)

    rot_sg = rot_pg1.copy() # create son genome (rotated)

    taken = rot_pg1[:sz] # elements copied from p1
    offset = sz # start copying the rest from p2 starting from the first locus out of the copied section
    idx2=0
    for i in range(offset,psz):
        while rot_pg2[idx2] in taken:
            idx2+=1
        rot_sg[i] = rot_pg2[idx2]
        idx2+=1

    son_genome = np.roll(rot_sg,rollnum)
    return Individual(son_genome)

def CX(parent1:Individual,parent2:Individual):
    pg1 = parent1.genome()
    pg2 = parent2.genome()
    psz = pg1.size
    
    son_genome = pg1.copy()
    
    pos_g1 = []
    id1=None
    while id1!=0:
        if id1 is None:
            id1=0
        pos_g1.append(id1)

        a2=pg2[id1] # p2 allele a2 corresponding to pg2[id1]
        id1=np.where(pg1==a2)[0][0] # index id1 such that pg1[id1]==a2 allele from p2

    taken = pg1[pos_g1]
    idx2=0
    pg2 = np.delete(pg2,pos_g1)
    for i in range(psz):
        if i not in pos_g1:
            son_genome[i] = pg2[idx2]
            idx2+=1

    return Individual(son_genome)    
